In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx

In [ ]:
ufs_nordeste = ['AL', 'BA', 'CE', 'MA', 'PB', 'PE', 'PI', 'RN', 'SE']

# Tratando PIB

In [ ]:
df_pib = pd.read_excel('PIB dos Municípios - base de dados 2010-2021.xlsx')
#print(df_pib_pc.head())

In [ ]:
df_pib = df_pib[df_pib['Ano'] == 2021][[
    'Sigla da Unidade da Federação',
    'Nome da Unidade da Federação',
    'Nome do Município',
    'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)',
    'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)'
]]

df_pib = df_pib[df_pib['Sigla da Unidade da Federação'].isin(ufs_nordeste)]


In [ ]:
df_pib = df_pib.rename(columns={
    'Sigla da Unidade da Federação': 'sigla_UF',
    'Nome da Unidade da Federação': 'nome_UF',
    'Nome do Município': 'nome_municipio',
    'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)': 'PIB',
    'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)': 'PIB_pc'
})
df_pib = df_pib.sort_values(by='PIB', ascending=False)

# Tratando rotas e PIB csv Recife

In [ ]:
df_recife = pd.read_csv('rotas_regic2021_brasil (Recife).csv')
#print(df_para_recife.head())

In [ ]:
df_recife = df_recife.drop(columns=['modal', 'cod_o', 'cod_uf_d'])
#print(df_recife.head())

In [ ]:
df_recife = df_recife[df_recife['cod_uf_o'].isin(ufs_nordeste)]
#print(df_nordeste.head())

In [ ]:
df_recife = df_recife.rename(columns={
    'cod_uf_o': 'sigla_UF',
    'nome_d': 'nome_UF',
    'nome_o': 'nome_municipio',
    'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)': 'PIB',
    'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)': 'PIB_pc'
})

In [ ]:
'''
df_final_recife = df_recife.merge(
    df_pib_pc[['Nome do Município', 'PIB']],
    left_on='nome_o',
    right_on='Nome do Município',
    how='left'
)
df_final_recife = df_final_recife.drop(columns=['Nome do Município'])
df_final_recife = df_final_recife.sort_values(by='PIB', ascending=False)
'''

# Tratando rotas e PIB csv Salvador

In [ ]:
df_salvador = pd.read_csv('rotas_regic2021_brasil (salvador).csv')

In [ ]:
df_salvador = df_salvador.drop(columns=['modal', 'cod_o', 'cod_uf_d'])

In [ ]:
df_salvador = df_salvador[df_salvador['cod_uf_o'].isin(ufs_nordeste)]

In [ ]:
df_salvador = df_salvador.rename(columns={
    'cod_uf_o': 'sigla_UF',
    'nome_d': 'nome_UF',
    'nome_o': 'nome_municipio',
    'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)': 'PIB',
    'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)': 'PIB_pc'
})

In [ ]:
'''
df_final_salvador = df_salvador.merge(
    df_pib_pc[['Nome do Município', 'PIB']],
    left_on='nome_o',
    right_on='Nome do Município',
    how='left'
)
df_final_salvador = df_final_salvador.drop(columns=['Nome do Município'])
df_final_salvador = df_final_salvador.sort_values(by='PIB', ascending=False)
'''

# Grafo

In [ ]:
# 1. ANÁLISE DE GRAFOS - Criando rede de distribuição
print("=== 1. ANÁLISE DE GRAFOS ===")

# Criar grafo da malha logística
G = nx.Graph()

# Adicionar nós (cidades) com atributos
for i, row in df_pib.iterrows():
    G.add_node(row['nome_municipio'], pib=row['PIB'])

for _, row in df_recife.iterrows():
    G.add_edge('Recife', row['nome_municipio'], weight=row['km'])  # peso numérico


In [ ]:
import matplotlib.pyplot as plt

pos = nx.spring_layout(G, seed=42)  # layout por mola
nx.draw_networkx(G, pos, node_size=400, font_size=8)  # nós e rótulos
plt.axis('off')
plt.show()



In [ ]:
#%pip install pyvis
from pyvis.network import Network

net = Network(height="600px", width="100%", notebook=True, directed=False)
net.from_nx(G)
net.show("rede.html")


# Extração Distâncias

In [ ]:
'''
import requests
import pandas as pd
import json
import time
import os

API_KEY = "eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6ImM3ZGEzZWU1ZWZhZDQyYzhiMTE2OWIzMDk4MDliMDBmIiwiaCI6Im11cm11cjY0In0="
URL_ROUTE = "https://api.openrouteservice.org/v2/directions/driving-car"
URL_GEOCODE = "https://api.openrouteservice.org/geocode/search"

# Carrega lista de cidades
df_cidades = pd.read_csv("lista_cidades.csv")  # ou pd.read_excel("lista_cidades.xlsx")
cidades = df_cidades["Cidade"].tolist()

resultados = []
backup_respostas = []

# Se já existir checkpoint, carregar progresso
if os.path.exists("checkpoint_resultados.json"):
    with open("checkpoint_resultados.json", "r", encoding="utf-8") as f:
        checkpoint = json.load(f)
        resultados = checkpoint["resultados"]
        backup_respostas = checkpoint["backup"]
        cidades_processadas = {(item["origem"], item["destino"]) for item in backup_respostas}
        print(f"▶ Retomando execução. {len(cidades_processadas)} rotas já processadas.")
else:
    cidades_processadas = set()

def geocode(texto):
    """Retorna coordenadas (lon, lat) de um endereço"""
    r = requests.get(URL_GEOCODE, params={"api_key": API_KEY, "text": texto})
    data = r.json()
    if "features" in data and len(data["features"]) > 0:
        return data["features"][0]["geometry"]["coordinates"]
    return None

def consulta_rota(origem, destino):
    """Consulta rota e retorna resposta completa + resumo"""
    origem_coord = geocode(origem)
    destino_coord = geocode(destino)

    if not origem_coord or not destino_coord:
        return None, None

    body = {"coordinates": [origem_coord, destino_coord]}
    headers = {"Authorization": API_KEY, "Content-Type": "application/json"}

    r = requests.post(URL_ROUTE, json=body, headers=headers)
    if r.status_code != 200:
        print(f"Erro: {origem} -> {destino}: {r.text}")
        return None, None

    data = r.json()
    try:
        summary = data["features"][0]["properties"]["summary"]
        distancia_km = summary["distance"] / 1000
        duracao_h = summary["duration"] / 3600
    except (KeyError, IndexError):
        return data, None

    resumo = {
        "origem": origem,
        "destino": destino,
        "distancia_km": round(distancia_km, 1),
        "duracao_h": round(duracao_h, 2),
    }
    return data, resumo

contador = 0
for cidade in cidades:
    for origem in ["Recife, PE", "Salvador, BA"]:
        if (origem, cidade) in cidades_processadas:
            continue

        data, resumo = consulta_rota(origem, cidade)
        if data:
            backup_respostas.append({"origem": origem, "destino": cidade, "resposta_completa": data})
        if resumo:
            resultados.append(resumo)

        contador += 1
        # Salvar checkpoint a cada 100 rotas
        if contador % 100 == 0:
            with open("checkpoint_resultados.json", "w", encoding="utf-8") as f:
                json.dump({"resultados": resultados, "backup": backup_respostas}, f, ensure_ascii=False, indent=2)
            print(f"💾 Checkpoint salvo ({contador} rotas).")

        time.sleep(1)  # respeita rate limit

# Salvar resultado final
with open("respostas_completas.json", "w", encoding="utf-8") as f:
    json.dump(backup_respostas, f, ensure_ascii=False, indent=2)

df_resultados = pd.DataFrame(resultados)
df_resultados.to_excel("distancias_resumo.xlsx", index=False)

# Remover checkpoint após concluir
if os.path.exists("checkpoint_resultados.json"):
    os.remove("checkpoint_resultados.json")

print("✅ Arquivos finais gerados: respostas_completas.json e distancias_resumo.xlsx")
'''

In [ ]:
import requests
import pandas as pd
import json
import time
import os

# Parâmetros
API_KEY = "eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6ImM3ZGEzZWU1ZWZhZDQyYzhiMTE2OWIzMDk4MDliMDBmIiwiaCI6Im11cm11cjY0In0=" #os.getenv("ORS_API_KEY")
print(API_KEY)
PROFILE = "driving-hgv"  # ou "driving-car"
URL_ROUTE = f"https://api.openrouteservice.org/v2/directions/{PROFILE}/geojson"

INPUT_FILE = "lista_cidades.csv"  # deve conter colunas: Cidade, lon, lat, PIB
OUT_BACKUP = "respostas_completas.json"
OUT_SUMMARY = "distancias_resumo.xlsx"

ORIGENS = ["Recife", "Salvador"]
PIB_LIMIAR = 300000  # R$ 300 mil
DAILY_CAP = 1900      # margem abaixo de 2.000
SLEEP_S = 1.6         # ~40/min

In [ ]:

# Carrega dados
if INPUT_FILE.lower().endswith(".xlsx"):
    df_cidades = pd.read_excel(INPUT_FILE)
else:
    df_cidades = pd.read_csv(INPUT_FILE)

# Normaliza nomes
df_cidades["Cidade"] = df_cidades["Cidade"].astype(str).str.strip()

#corrige moeda
df_cidades['PIB'] = (
    df_cidades['PIB']
    .astype(str)
    .str.replace('$', '', regex=False)
    .str.replace('.', '', regex=False)   # remove separador de milhar
    .str.replace(',', '.', regex=False)  # troca vírgula por ponto decimal
    .str.strip()
    .astype(float)
)

# Valida colunas
for col in ["Cidade", "lon", "lat", "PIB"]:
    if col not in df_cidades.columns:
        raise ValueError(f"Coluna obrigatória ausente: {col}")

# Filtra apenas as cidades acima do limiar de PIB (mantendo somente as 534 desejadas)
df_filtrado = df_cidades[df_cidades["PIB"] > PIB_LIMIAR].copy()

# Mapa nome -> [lon, lat]
coords = {r["Cidade"]: [float(r["lon"]), float(r["lat"])] for _, r in df_cidades.iterrows()}

# Garante que as origens estão presentes na tabela
for o in ORIGENS:
    if o not in coords:
        raise ValueError(f"Origem ausente na tabela: {o}")

In [ ]:

resultados = []
backup = []

session = requests.Session()
session.headers.update({"Authorization": API_KEY, "Content-Type": "application/json"})

def consulta_rota(origem_nome, destino_nome):
    o = coords.get(origem_nome)
    d = coords.get(destino_nome)
    if not o or not d:
        return None, None
    body = {"coordinates": [o, d]}
    r = session.post(URL_ROUTE, json=body, timeout=60)
    # Reação simples a 429: aguarda e tenta 1x
    if r.status_code == 429:
        time.sleep(10)
        r = session.post(URL_ROUTE, json=body, timeout=60)
    if r.status_code != 200:
        print(f"[ERRO {r.status_code}] {origem_nome} -> {destino_nome}: {r.text[:200]}")
        return None, None
    data = r.json()
    try:
        feat0 = data["features"]
        props = feat0["properties"]
        summary = props["summary"]
        dist_km = float(summary["distance"]) / 1000.0
        dur_h = float(summary["duration"]) / 3600.0
        resumo = {
            "origem": origem_nome,
            "destino": destino_nome,
            "origem_lon": o, "origem_lat": o[21],
            "destino_lon": d, "destino_lat": d[21],
            "perfil": PROFILE,
            "distancia_km": round(dist_km, 1),
            "duracao_h": round(dur_h, 2),
        }
    except (KeyError, IndexError, TypeError, ValueError):
        resumo = None
    return data, resumo

contador = 0
for _, row in df_filtrado.iterrows():
    cidade = row["Cidade"]
    for origem in ORIGENS:
        if contador >= DAILY_CAP:
            print(f"[INFO] Limite diário interno atingido ({DAILY_CAP}). Encerrando.")
            break
        data, resumo = consulta_rota(origem, cidade)
        if data:
            backup.append({"origem": origem, "destino": cidade, "resposta_completa": data})
        if resumo:
            resultados.append(resumo)
        contador += 1
        time.sleep(SLEEP_S)
    if contador >= DAILY_CAP:
        break

# Salva resultados
with open(OUT_BACKUP, "w", encoding="utf-8") as f:
    json.dump(backup, f, ensure_ascii=False, indent=2)

pd.DataFrame(resultados).to_excel(OUT_SUMMARY, index=False)

print(f"✅ Feito. Rotas: {len(resultados)} | Arquivos: {OUT_BACKUP}, {OUT_SUMMARY}")


## Read .json backup

In [ ]:
import json
import pandas as pd

OUT_BACKUP = "raw_data/respostas_completas.json"

with open(OUT_BACKUP, "r", encoding="utf-8") as f:
    dados = json.load(f)

resumo_list = []
for item in dados:
    origem = item.get("origem")
    destino = item.get("destino")
    resposta = item.get("resposta_completa", {})
    try:
        summary = resposta["features"][0]["properties"]["summary"]
        coordinates = resposta["metadata"]["query"]["coordinates"]
        lat = coordinates[1][1]
        lon = coordinates[1][0]
        distancia_km = summary["distance"] / 1000
        duracao_h = summary["duration"] / 3600
    except (KeyError, IndexError, TypeError):
        distancia_km = None
        duracao_h = None
    resumo_list.append({
        "origem": origem,
        "destino": destino,
        "origem_lat": lat,
        "origem_lon": lon,
        "distancia_km": distancia_km,
        "duracao_h": duracao_h
    })

df_resumo = pd.DataFrame(resumo_list)

# Limpeza Municipios e contrução tabela final

In [ ]:
#%pip install pandas xlrd
import pandas as pd

# Caminho do arquivo .xls (use raw string para evitar problemas com backslashes no Windows)
caminho = r"RELATORIO_DTB_BRASIL_2024_MUNICIPIOS.xls"
nome_planilha = "DTB_Municípios"  # equivalente ao item [Name="DTB_Municípios"] no Power Query

# 1) Ler a planilha sem cabeçalho (equivalente a ter Column1..Column17) e como texto inicialmente
#    Para .xls, o engine 'xlrd' pode ser necessário (instale xlrd se não estiver presente)
df_cidades = pd.read_excel(
    caminho,
    sheet_name=nome_planilha,
    header=None,
    dtype=str,
    engine="xlrd"  # remova ou ajuste se não for necessário no seu ambiente
)

# 2) Remover as 6 primeiras linhas (Table.Skip(...,6))
df_cidades = df_cidades.iloc[6:].reset_index(drop=True)

# 3) Promover a primeira linha a cabeçalho (Table.PromoteHeaders)
df_cidades.columns = df_cidades.iloc[0].astype(str).str.strip()
df_cidades = df_cidades.iloc[1:].reset_index(drop=True)

# 4) Definir os tipos conforme o Power Query
#    Inteiros (usando dtype pandas 'Int64' que aceita NA) e textos (string Pandas)
colunas_inteiras = [
    "UF",
    "Região Geográfica Intermediária",
    "Nome Região Geográfica Intermediária",  # esta fica texto; listada aqui só para referência
    "Região Geográfica Imediata",
    "Nome Região Geográfica Imediata",       # esta fica texto; listada aqui só para referência
    "Município",
    "Código Município Completo",
]
# Converter explicitamente apenas as que são numéricas
cols_num = ["UF", "Região Geográfica Intermediária", "Região Geográfica Imediata", "Município", "Código Município Completo"]
for c in cols_num:
    if c in df_cidades.columns:
        df_cidades[c] = pd.to_numeric(df_cidades[c], errors="coerce").astype("Int64")

# Colunas de texto relevantes
cols_texto = [
    "Nome_UF",
    "Nome Região Geográfica Intermediária",
    "Nome Região Geográfica Imediata",
    "Nome_Município",
]
for c in cols_texto:
    if c in df_cidades.columns:
        df_cidades[c] = df_cidades[c].astype("string")

# 5) Selecionar somente as colunas desejadas (Table.SelectColumns do M)
colunas_desejadas = [
    "UF",
    "Nome_UF",
    "Região Geográfica Intermediária",
    "Nome Região Geográfica Intermediária",
    "Região Geográfica Imediata",
    "Nome Região Geográfica Imediata",
    "Município",
    "Código Município Completo",
    "Nome_Município",
]
df_cidades = df_cidades[[c for c in colunas_desejadas if c in df_cidades.columns]]

# 6) Remover as colunas de regiões (Table.RemoveColumns do M)
colunas_remover = [
    "Região Geográfica Intermediária",
    "Nome Região Geográfica Intermediária",
    "Região Geográfica Imediata",
    "Nome Região Geográfica Imediata",
]
df_cidades = df_cidades.drop(columns=[c for c in colunas_remover if c in df_cidades.columns])

# 7) Remover duplicatas (Table.Distinct)
df_cidades = df_cidades.drop_duplicates().reset_index(drop=True)

# 8) Filtrar apenas estados do Nordeste (Table.SelectRows com OR em Nome_UF)
estados_ne = [
    "Alagoas", "Bahia", "Ceará", "Maranhão", "Paraíba",
    "Pernambuco", "Piauí", "Rio Grande do Norte", "Sergipe"
]
if "Nome_UF" in df_cidades.columns:
    df_cidades = df_cidades[df_cidades["Nome_UF"].isin(estados_ne)].reset_index(drop=True)

# 9) (Opcional) Exportar o resultado
# df.to_csv("DTB_Municipios_NE_2024.csv", index=False)

# Para inspeção rápida:
print(df_cidades.head())
print(df_cidades.dtypes)

In [ ]:
import pandas as pd
import numpy as np

# Arquivos de entrada (no mesmo diretório)
coords = pd.read_csv('coordenadas_e_pib_cidades.csv')
dist = pd.read_csv('distancia_e_tempo_cidades.csv')

# Normaliza strings
for col in ['origem','destino']:
    dist[col] = dist[col].astype(str).str.strip()
coords['Cidade'] = coords['Cidade'].astype(str).str.strip()

# Converte PIB "$73436128,432" -> 73436128.432 (float)
def parse_pib(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    s = s.replace('"','').replace("'", '')
    s = s.replace('$','').replace(' ','')
    s = s.replace('.', '')       # remove separador de milhar (se houver)
    s = s.replace(',', '.')      # vírgula -> ponto
    try:
        return float(s)
    except:
        return np.nan

coords['PIB'] = coords['PIB'].apply(parse_pib)

# Zera self-distances ausentes (origem == destino)
mask_self = (dist['origem'] == dist['destino'])
dist.loc[mask_self & dist['distancia_km'].isna(), 'distancia_km'] = 0.0
dist.loc[mask_self & dist['duracao_h'].isna(), 'duracao_h'] = 0.0

# Converte duração para minutos
dist['duracao_min'] = pd.to_numeric(dist['duracao_h'], errors='coerce') * 60

# Tabela Recife
recife = dist[dist['origem'] == 'Recife'][['destino','distancia_km','duracao_min']].rename(
    columns={'destino':'Cidade','distancia_km':'dist_recife_km','duracao_min':'tempo_recife_min'}
)

# Tabela Salvador
salvador = dist[dist['origem'] == 'Salvador'][['destino','distancia_km','duracao_min']].rename(
    columns={'destino':'Cidade','distancia_km':'dist_salvador_km','duracao_min':'tempo_salvador_min'}
)

# Merge final
merged = coords.merge(recife, on='Cidade', how='left').merge(salvador, on='Cidade', how='left')

# Renomeia e organiza colunas conforme o template
merged = merged.rename(columns={'Cidade':'municipio', 'PIB':'pib'})

# Adiciona coluna 'estado' buscando pelo município no arquivo cod_cidades.csv
df_cod = pd.read_csv('cod_cidades.csv')
df_cod['Nome_Município'] = df_cod['Nome_Município'].astype(str).str.strip()
df_cod['Nome_UF'] = df_cod['Nome_UF'].astype(str).str.strip()

# Faz o merge para obter o estado correspondente ao município
merged = merged.merge(df_cod[['Nome_Município', 'Nome_UF']], left_on='municipio', right_on='Nome_Município', how='left')
merged['estado'] = merged['Nome_UF']
merged = merged.drop(columns=['Nome_Município', 'Nome_UF'])

cols = ['municipio','estado','lat','lon','pib',
        'dist_recife_km','dist_salvador_km','tempo_recife_min','tempo_salvador_min']
merged = merged[cols]

# Arredonda e salva
num_cols = ['lat','lon','pib','dist_recife_km','dist_salvador_km','tempo_recife_min','tempo_salvador_min']
merged[num_cols] = merged[num_cols].round(6)



# Filtra apenas cidades com PIB > 300.000
merged = merged[merged['pib'] > 300000].copy()

# Remove linhas com dados faltantes nas colunas de distância
merged = merged.dropna(subset=['dist_recife_km', 'dist_salvador_km'])

merged.to_csv('dados_nordeste_merged.csv', index=False)
print('Arquivo gerado: dados_nordeste_merged.csv')

In [ ]:
df_dados_nordeste = pd.read_csv('dados_nordeste_merged.csv')

df_dados_nordeste.head(20)